<center><img src="images/header.png"></center>

<h1><center>Лекция №6: Введение в анализ социальных сетей</center></h1>
<hr>
<h1><center>Домашняя работа</center></h1>

Данное ДЗ является довольно творческим и оно требует довольно много самостоятельной работы, так что не стоит с ним мешкать =)

Вам предлагается выбрать сообщество (или сообщества) в сети ВКонтакте и проанализировать его (их) контингент. 


**Правила:**
* Отчеты принимаются до **23:59 11 ноября**. Вы должны залить ваш ноутбук в github дать на него ссылку в следующей [форме](https://forms.gle/88d9CfaJN5gPV4bB8).
* Работоспособность будет проверяться через [colab](https://colab.research.google.com) (можете сразу давать ссылку на него). Если ноутбук не будет исполняться (падать по ошибке) то все, что будет написано после ошибки не будет учитываться при формировании оценки 
* Вы должны включить код, с помощью которого вы общались с API вконтакте, но для экономии времени, заранее предусмотреть возможность загрузки уже собранных данных (например, как это было реализовано в ноутбуках по кластеризации через `wget` и dropbox или raw данными в github)

**Пожелания:**
* Выбирайте для анализа небольшие сообщества (до 5к), желательно чтобы вы сами в них состояли - это ускорит вычисления и упростит интерпретацию
* Отнеситесь к этой работе как к исследованию - больше графиков и комментариев!
* Красивые визуализации лучше делать через gephi!

# Загрузка данных (2 балла)

Изучите следующие материалы
* [Знакомство с API](https://vk.com/dev/first_guide)
* [Туториал](https://nbviewer.jupyter.org/github/allatambov/Py-programming-3/blob/master/15-06/lect-vk-api.ipynb) по выгрузке данных с помощью VKAPI
* Про метод execute - [туть](https://vk.com/dev/execute) и [туть](http://www.cyberforum.ru/python-web/thread1730446.html)

Выберите сообщество и загрузите
* Идентификаторы всех [участников сообщества](https://vk.com/dev/groups.getMembers)
* [Списки друзей](https://vk.com/dev/friends.get) участников и их признаки (пол, возраст, университет и все, что посчитаете нужным - нам потом группировать по этим атрибутам)
* [Пересечение](https://vk.com/dev/friends.getMutual) между списками друзей

Таким образом вы должны получить замкнутую группу аккаунтов с их друзьями и связями между ними.

Далее - соберите их этого граф в networkX и сохраните в graphml или gml формате

**Замечания:**
* Вконтакте может ограничить вам скорость запросов, поэтому иногда имеет смысл ставить небольшую паузу (`time.sleep(1)`) между запросами и сохранять предварительные результаты
* Код выгрузки может быть написан как с помощью модуля `vk` так и просто с помощью requests 


In [1]:
### Здесь должен быть код загрузки из VK
### Его можно закомментировать
import vk                                                                       
import time                                                                     
import pickle                                                                   
                                                                                
def save_obj(obj, name ):                                                       
    with open(name + '.pkl', 'wb') as f:                                        
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)                            
                                                                                
def load_obj(name ):                                                            
    with open(name + '.pkl', 'rb') as f:                                        
        return pickle.load(f)                                                   

"""tok = '--------'
                                                                    
session = vk.Session(access_token = tok) # открыть сессию для работы            
api = vk.API(session) # подключиться к API                                      
                                                                                
group = 'ffmsu216'                                                              
d = api.groups.getMembers(group_id=group, fields = ['sex', 'bdate', 'city', 'country', 'lists', 'domain', 'has_mobile', 'contacts', 'connections', 'site', 'education', 'universities', 'schools', 'can_post', 'can_see_all_posts', 'can_see_audio', 'can_write_private_message', 'status', 'last_seen', 'common_count', 'relation', 'relatives'],v='5.103')
all_members = d['items']                                                        

count = 0                                                                       
for member in all_members:                                                      
    count = count+1                                                             
    print(count)                                                                
    while True:                                                                 
        try:                                                                    
            member['friends'] = api.friends.get(user_id=member['id'],fields=['nickname', 'domain', 'sex', 'bdate', 'city', 'country', 'timezone', 'has_mobile', 'contacts', 'education', 'online', 'relation', 'last_seen', 'status', 'can_write_private_message', 'can_see_all_posts', 'can_post', 'universities'],v='5.103')
        except Exception as e:                                                  
            if e.__dict__['error_data']['error_code'] == 30:                    
                member['friends'] = 'private'                                   
            elif e.__dict__['error_data']['error_code'] == 6:                   
                time.sleep(0.4)                                                 
                continue                                                        
            else:                                                               
                member['friends'] = 'unknown'                                   
            break                                                               
        break    
all_members = [member for member in all_members if not isinstance(member['friends'], str)]
all_members = [member for member in all_members if member['friends']['count'] < 1100]
for member in all_members:                                                      
    member.update({'all_friends_checked':False})                                
    if not isinstance(member['friends'], str):                                  
        for member_friend in member['friends']['items']:                        
            member_friend['common_fr'] = []                                     
count = 0                                                                       
for member in all_members:                                                      
    if isinstance(member['friends'], str) or member['can_access_closed'] == False:
        continue                                                                
    count +=1                                                                   
    print(count)                                                                
    count2 =0                                                                   
    for member_friend in member['friends']['items']:                            
        if 'is_closed' not in member_friend.keys() or member_friend['is_closed'] == True:
            continue                                                            
        count2 +=1                                                              
        print('     ',count2, count)                                            
        member_id = []                                                          
        for member2 in all_members:                                             
            if not member2['all_friends_checked'] and member2['friends']!='unknown' and member2['friends']!='private':
                member_id.append(member2['id'])                                 
            if len(member_id) == 100:                                           
                time.sleep(0.1)                                                 
                while True:                                                     
                    try:                                                        
                        common_fr = api.friends.getMutual(source_uid=member_friend['id'],target_uids=member_id,v='5.103')
                    except Exception as e:                                      
                        if e.__dict__['error_data']['error_code'] == 6:         
                            time.sleep(0.4)                                     
                            print('sleep')                                      
                            continue                                            
                        else:                                                   
                            print('UNKONW',str(e))                              
                        break                                                   
                    break                                                       
                member_friend['common_fr'] += common_fr                         
                member_id.clear()                                               
        if len(member_id) > 0:                                                  
            time.sleep(0.1)                                                     
            while True:                                                         
                try:                                                            
                    common_fr = api.friends.getMutual(source_uid=member_friend['id'],target_uids=member_id,v='5.103')
                except Exception as e:                                          
                    if e.__dict__['error_data']['error_code'] == 6:             
                        time.sleep(0.4)                                         
                        print('sleep')                                          
                        continue                                                
                    else:                                                       
                        print('UNKONW',str(e))                                  
                    break                                                       
                break                                                           
            member_friend['common_fr'] += common_fr                             
            member_id.clear()                                                   
    member['all_friends_checked']=True                                          
#save_obj(all_members,'all_members_with_friends_with_friends21')                 
print('Saved exit')   """                                                          


"tok = '--------'\n                                                                    \nsession = vk.Session(access_token = tok) # открыть сессию для работы            \napi = vk.API(session) # подключиться к API                                      \n                                                                                \ngroup = 'ffmsu216'                                                              \nd = api.groups.getMembers(group_id=group, fields = ['sex', 'bdate', 'city', 'country', 'lists', 'domain', 'has_mobile', 'contacts', 'connections', 'site', 'education', 'universities', 'schools', 'can_post', 'can_see_all_posts', 'can_see_audio', 'can_write_private_message', 'status', 'last_seen', 'common_count', 'relation', 'relatives'],v='5.103')\nall_members = d['items']                                                        \n\ncount = 0                                                                       \nfor member in all_members:                                         

In [6]:
### Здесь должен быть код преобразования в граф и сохранение в файл
### Его тоже можно закомментировать
import networkx as nx

"""all_members = load_obj('./vk/all_members_with_friends_with_friends21')               
g = nx.Graph()                                                                  
for member in all_members:                                                      
    g.add_nodes_from([(member['id'], {                                          
        'sex':member.get('sex',-1),                                             
        'first_name':member.get('first_name','unknown'),                        
        'last_name':member.get('last_name','unknown'),                          
        'country':member.get('country',{}).get('title', 'unknown'),             
        'city':member.get('city',{}).get('title', 'unknown'),                   
        'university_name':member.get('university_name','unknown')               
        })])                                                                    
    print(g.number_of_nodes())                                                  
    if isinstance(member['friends'], str) or member['can_access_closed'] == False:
        continue                                                                
    g.add_nodes_from((friend['id'], {                                           
        'sex':friend.get('sex',-1),                                             
        'first_name':friend.get('first_name','unknown'),                        
        'last_name':friend.get('last_name','unknown'),                          
        'city':friend.get('city',{}).get('title', 'unknown'),                   
        'university_name':friend.get('university_name','unknown')               
        }) for friend in member['friends']['items'])                            
    print(member['last_name'], member['friends']['count'],g.number_of_nodes())  
    g.add_edges_from((member['id'],friend['id']) for friend in member['friends']['items'])
for member in all_members:                                                      
    if isinstance(member['friends'], str) or member['can_access_closed'] == False:
        continue                                                                
    for friend in member['friends']['items']:                                   
        for cm in friend['common_fr']:                                          
            g.add_edges_from((friend['id'],id_) for id_ in cm['common_friends'])

g=nx.k_core(g,k=35)
print(g.nodes())                                                         
nx.set_node_attributes(g,nx.betweenness_centrality(g),'betw_cent')             
nx.set_node_attributes(g,nx.closeness_centrality(g),'close_cent')              
nx.write_graphml(g, 'vk_graph_pres.graphml')                                   
print(g.number_of_nodes())                                                      
print(g.number_of_edges())"""                                                                                                        


"all_members = load_obj('./vk/all_members_with_friends_with_friends21')               \ng = nx.Graph()                                                                  \nfor member in all_members:                                                      \n    g.add_nodes_from([(member['id'], {                                          \n        'sex':member.get('sex',-1),                                             \n        'first_name':member.get('first_name','unknown'),                        \n        'last_name':member.get('last_name','unknown'),                          \n        'country':member.get('country',{}).get('title', 'unknown'),             \n        'city':member.get('city',{}).get('title', 'unknown'),                   \n        'university_name':member.get('university_name','unknown')               \n        })])                                                                    \n    print(g.number_of_nodes())                                                  \n    if isi

In [7]:
### Здесь код, который загружает сохраненную сеть
g = nx.read_graphml('vk_graph_with_comm_is_member.graphml')

# Визуализация (1 балл)

* Откройте полученный граф в gephi
* Выберите наиболее удачный на ваш взгляд алгоритм прорисовки
* Добавьте изменение размера вершины в зависимости от степени вершины
* Вставьте скрин-шот ниже

### Ваши картинки и комментарии ниже
Группа физфака №116 (degree ranking)

nodes: 4709; 
edges: 92309; 

<img src='./images/all_group.png'/>


# Центральности вершин (1.5 балла)

Возможно, перед тем как пойти дальше, стоит немного почитстить граф - с помощью [kcore](https://www.google.com/search?client=firefox-b-d&q=networkx+kcore)

* Найдите наиболее влиятельные вершины с точки зрения degree, closeness и betweenness centrality
* Выделите их на картинке в gephi

In [9]:
#код с использованием kcore в разделе создания графа^^^
#g=nx.k_core(g,k=35)                                                       
#nx.set_node_attributes(g,nx.betweenness_centrality(g),'betw_cent')             
#nx.set_node_attributes(g,nx.closeness_centrality(g),'close_cent') 

### Ваш код, картинки и комментарии ниже
Closeness centrality
<img src='./images/close.png'/>
Betweenness centrality
<img src='./images/betw.png'/>

На обоих графах выделяется наш груповод, что логично

# Сообщества... в сообществе (1.5 балла)

Выберите алгоритм выделения сообществ на графах и алгоритм на основе похожести вершин. Для обоих вариантов
* Выделите сообщества, опираясь на максимальное значение модулярности для полученного разбиения
* Изобразите граф с вершинами, раскрашенными в цвет своего сообщества в gephi
* Попробуйте проинтерпретировать полученные сообщества, кто эти люди, почему они всязаны?
* Какой метод на ваш взгляд сработал лучше?

In [11]:
### Ваш код, картинки и комментарии ниже
#Label Propagation
lp_partition = nx.community.asyn_lpa_communities(g,seed=123) # должен быть seed
lp_partition = list(lp_partition) # разбиение на сообщества 
print([len(i) for i in lp_partition])

[761, 99, 153]


In [12]:
some_attributes = {}
for label, cluster in enumerate(lp_partition):
    some_attributes.update({i:label for i in cluster})
nx.set_node_attributes(g, some_attributes, 'comunities')#добавляем атрибут, номер сообщества
nx.write_graphml(g, 'vk_graph_with_comm_pres.graphml')

Label Propogation
<img src='./images/label_prop2.png'/>
Зеленая группа - ребята олимпиадники, они связаны с обычными людьми тремя организторами олимпиад, один из которых -  член группы

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Все положительные похожести по Жаккару
# По похожести
g = nx.convert_node_labels_to_integers(g,0)
jac = nx.jaccard_coefficient(g)
jac = np.array(list(jac))
print(g.number_of_nodes())

In [16]:
# Немного магии, чтобы получить из этого матрицу попарных СХОЖЕСТЕЙ по жаккару
from scipy.sparse import coo_matrix

# Переводим все в формат разреженной матрицы
D = coo_matrix((jac[:,2], (jac[:,0], jac[:,1])), shape=(1013,1013))

# Делаем ее симметричной
D = D+D.T

In [17]:
# Попробуем посчитать на этой близости спектральную кластеризацию
from sklearn.cluster import SpectralClustering

In [18]:
model = SpectralClustering(n_clusters=4, affinity='precomputed')#что заранее сделали матрицу похожести
model.fit(D)


SpectralClustering(affinity='precomputed', assign_labels='kmeans', coef0=1,
                   degree=3, eigen_solver=None, eigen_tol=0.0, gamma=1.0,
                   kernel_params=None, n_clusters=4, n_init=10, n_jobs=None,
                   n_neighbors=10, random_state=None)

In [31]:
labels = list(model.labels_)

In [32]:

nx.set_node_attributes(g,{node:label for label, node in zip(labels,g.nodes) }, 'commutity_jac') 
nx.write_graphml(g, 'vk_graph_with_comm_and jac_pres.graphml')

По Жаккару
<img src='./images/jac.png'/>
В этом случае 2 организатора олимпиад распределены в другие группы, это странно

# Асортативность по признакам (1.5 балла)

* Посчитайте коэффициенты асортативности для признаков вершин (пол, возраст, университет, школа, город, количество друзей, etc)
* Проинтерпретируйте полученные результаты

In [9]:
### Ваш код, картинки и комментарии ниже